In [202]:
import dataRead
import dataClean
import runTrain
import runTest
import writeResult

In [251]:
# Read data
reload(dataRead)
X_train,X_test,word_vecs = dataRead.do()

In [256]:
# Test Above function here


numpy.float64

In [322]:
reload(dataClean)
X_train_features,X_test_features = dataClean.clean(X_train,X_test)
X_train_features,X_test_features = dataClean.getWordVectors(X_train_features,X_test_features,word_vecs)
X_train_features,X_test_features = dataClean.getSentenceVectors(X_train_features,X_test_features)

In [325]:
# Test Above function here


In [55]:
# Train model
reload(runTrain)
runTrain.do(X_train_features)

In [6]:
# Test Above function here


In [68]:
# Test model
reload(runTest)
result = runTest.do(X_test_features)

In [69]:
# Test Above function here


In [70]:
# Write Results
reload(writeResult)
writeResult.do(result,X_test_features)

In [ ]:
# Test Above function here